In [93]:
import pandas as pd

In [94]:
df = pb.read_csv("Tweets2.csv")
# df.head()

In [95]:
review_df = df[['text','airline_sentiment']]

# review_df = review_df[review_df['airline_sentiment'] != 'neutral']

review_df["airline_sentiment"].value_counts()

# df = review_df['airline_sentiment']

negative    3507
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [96]:
sentiment_label = review_df.airline_sentiment.factorize()

In [97]:
tweet = review_df.text.values

In [98]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweet)

In [99]:
encoded_docs = tokenizer.texts_to_sequences(tweet)

In [100]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [101]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding
embedding_vector_length = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(100, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 200, 32)           423488    
                                                                 
 spatial_dropout1d_5 (Spatia  (None, 200, 32)          0         
 lDropout1D)                                                     
                                                                 
 lstm_5 (LSTM)               (None, 100)               53200     
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 476,789
Trainable params: 476,789
Non-trainable params: 0
________________________________________________

In [102]:
history = model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=20, batch_size=32)

Epoch 1/20
225/225 [==============================] - 22s 93ms/step - loss: 0.3525 - accuracy: 0.3047 - val_loss: -1.3154 - val_accuracy: 0.1020
Epoch 2/20
225/225 [==============================] - 26s 114ms/step - loss: -0.8349 - accuracy: 0.3547 - val_loss: -12.9642 - val_accuracy: 0.1416
Epoch 3/20
225/225 [==============================] - 25s 113ms/step - loss: -2.9808 - accuracy: 0.4105 - val_loss: -22.7463 - val_accuracy: 0.1276
Epoch 4/20
225/225 [==============================] - 25s 113ms/step - loss: -10.9046 - accuracy: 0.4831 - val_loss: -40.9847 - val_accuracy: 0.1187
Epoch 5/20
225/225 [==============================] - 25s 113ms/step - loss: -16.9591 - accuracy: 0.4931 - val_loss: -55.3326 - val_accuracy: 0.1321
Epoch 6/20
225/225 [==============================] - 27s 121ms/step - loss: -23.4319 - accuracy: 0.5063 - val_loss: -69.7056 - val_accuracy: 0.1427
Epoch 7/20
225/225 [==============================] - 31s 136ms/step - loss: -29.9220 - accuracy: 0.5098 - val_l

In [103]:
def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw,maxlen=200)
    prediction = int(model.predict(tw).round().item())
    print("Predicted label: ", sentiment_label[1][prediction])
test_sentence1 = "I enjoyed my journey on this flight."
predict_sentiment(test_sentence1)
test_sentence2 = "This is the worst flight experience of my life!"
predict_sentiment(test_sentence2)

Predicted label:  positive
Predicted label:  positive


In [106]:
predict_sentiment("He is a bad boy")

Predicted label:  positive
